In [1]:
import torch
import numpy as np

## Etude 1
Let $u$ be a vector of $2D$ points. We need to get a matrix $A$ whose element $(i,j)$ depends on euclidean distance between $u_i$ and $u_j$.

$$u \in \mathbb{R}^{N \times 2}$$
$$A \in \mathbb{R}^{N \times N}, \ A_{ij} = f(\|u_i - u_j\|) $$

Particularly, let's take $f(x) = exp(-\frac{x}{4})$.

### Naive approach

In [2]:
def naive_etude1(u, N):
    result = torch.zeros(N, N)
    for i in range(N):
        for j in range(N):
            result[i,j] = torch.exp(-0.25 * torch.dist(u[i], u[j]))
    return result

### Vectorized approaches

In [3]:
def fully_vectorized_etude1(u, N):
    u_ = u.reshape((1,) + tuple(u.shape)) # add one dimension for transpose()
    A_ = u_.numpy().transpose((1,0,2)) - u_ # row - column: broadcasting subtraction
    return torch.exp(-0.25 * torch.sqrt(A_[:,:,0]**2 + A_[:,:,1]**2)) 

In [4]:
def one_dim_vectorized_etude1(u, N):
    result = torch.zeros(N, N)
    for i in range(N):
        du = u[i] - u
        result[i,:] = torch.exp(-0.25 * torch.sqrt(du[:,0]**2 + du[:,1]**2))
    return result

### Comparison

#### fully vectorized vs. naive

In [5]:
N = 10**3
u = torch.randn(N, 2)

In [6]:
%%time
A = naive_etude1(u, N)

CPU times: user 12.3 s, sys: 376 ms, total: 12.7 s
Wall time: 12.7 s


In [7]:
%%time
B = fully_vectorized_etude1(u, N)

CPU times: user 20 ms, sys: 8 ms, total: 28 ms
Wall time: 22.1 ms


In [8]:
torch.equal(A, B)

True

#### 1-dim vectorized vs. fully vectorized

In [9]:
N = 10**4
u = torch.randn(N, 2)

In [10]:
%%timeit
A = fully_vectorized_etude1(u, N)

1.91 s ± 161 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [11]:
%%timeit
B = one_dim_vectorized_etude1(u, N)

1.57 s ± 18 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [12]:
torch.equal(A, B)

True